In [1]:
import numpy as np
import torch
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader


In [2]:
def preprocess_data(csv_file, sequence_length):
    
  
    df = pd.read_csv(csv_file)
    
    features = df[['magnitude', 'cdi', 'mmi', 'dmin', 'gap', 'depth']].values
    
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(features)
    
    sequences = []
    for i in range(0, len(normalized_features) - sequence_length + 1, sequence_length // 2):  
        sequence = normalized_features[i:i + sequence_length]
        sequences.append(sequence)
    
    return torch.FloatTensor(sequences), scaler

In [3]:
normal_sequences, normal_scaler = preprocess_data("Normal.csv", 50)
anomalous_sequences, anomalous_scaler = preprocess_data("Anomalous.csv", 50)


/tmp/ipykernel_6002/2731436145.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400268359/work/torch/csrc/utils/tensor_new.cpp:261.)
  return torch.FloatTensor(sequences), scaler


In [4]:

from backup import TsunamiDetector


In [5]:
# Initialize the model
sequence_length = 50  # Sequence length for training (you already have it)
latent_dim = 100  # Latent dimension for the generator (can be adjusted)
lambda_ano = 0.1 # Weight for anomaly loss
num_features = 6 # Number of features (as in the input data)


# Create the TsunamiDetector instance
tsunami_detector = TsunamiDetector(sequence_length, latent_dim, lambda_ano, num_features,device='cpu')

# Train the model on the normal sequences (normal_sequences is already preprocessed)
batch_size = 64
epochs = 200

# Convert normal_sequences to a DataLoader compatible format (TensorDataset)
normal_data = TensorDataset(normal_sequences)
normal_dataloader = DataLoader(normal_data, batch_size=batch_size, shuffle=True)

# Train the model
tsunami_detector.train(normal_dataloader, epochs=epochs)



Epoch 0: G_loss: 0.8808, D_loss: 0.6847
Epoch 1: G_loss: 0.7845, D_loss: 0.7098
Epoch 2: G_loss: 0.6776, D_loss: 0.7235
Epoch 3: G_loss: 0.6490, D_loss: 0.7632
Epoch 4: G_loss: 0.5771, D_loss: 0.7963
Epoch 5: G_loss: 0.5646, D_loss: 0.7927
Epoch 6: G_loss: 0.5760, D_loss: 0.7795
Epoch 7: G_loss: 0.5969, D_loss: 0.7720
Epoch 8: G_loss: 0.5908, D_loss: 0.7704
Epoch 9: G_loss: 0.5904, D_loss: 0.7327
Epoch 10: G_loss: 0.6050, D_loss: 0.7268
Epoch 11: G_loss: 0.6575, D_loss: 0.7142
Epoch 12: G_loss: 0.6799, D_loss: 0.6765
Epoch 13: G_loss: 0.6984, D_loss: 0.7019
Epoch 14: G_loss: 0.6762, D_loss: 0.7209
Epoch 15: G_loss: 0.6863, D_loss: 0.7089
Epoch 16: G_loss: 0.7015, D_loss: 0.6934
Epoch 17: G_loss: 0.6972, D_loss: 0.7000
Epoch 18: G_loss: 0.7353, D_loss: 0.6398
Epoch 19: G_loss: 0.7047, D_loss: 0.6697
Epoch 20: G_loss: 0.7783, D_loss: 0.6549
Epoch 21: G_loss: 0.7804, D_loss: 0.6667
Epoch 22: G_loss: 0.7917, D_loss: 0.6290
Epoch 23: G_loss: 0.7948, D_loss: 0.6317
Epoch 24: G_loss: 0.7764, 

In [6]:
# Calibrate threshold with normal data
validation_data = normal_sequences  # You can use a validation set or a portion of normal data
threshold_percentile = 95
threshold = tsunami_detector.calibrate_threshold(validation_data, percentile=threshold_percentile)

print(f"Calibrated threshold: {threshold}")


Calibrated threshold: 3.057793378829956


In [7]:
# Assuming anomalous_sequences is a tensor of anomalous data (like actual tsunami events)
anomalous_sequences = torch.FloatTensor(anomalous_sequences)  # Ensure it's a tensor if not already

# Evaluate anomalous data
for sequence in anomalous_sequences:
    result = tsunami_detector.detect_tsunami(sequence)
    print(f"ANOMALY: Is Tsunami: {result['is_tsunami']}, Confidence: {result['confidence']}, Anomaly Score: {result['anomaly_score']}")


ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.86704421043396
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.2189674377441406
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.8718364238739014
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.5545028448104858
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.8520749807357788
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.9115089178085327
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.775671362876892
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.5681381225585938
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.7589538097381592
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.6434121131896973
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.8370535373687744
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.14249324798584


In [8]:
normal_sequences = torch.FloatTensor(normal_sequences)  # Ensure it's a tensor if not already

# Evaluate anomalous data
for sequence in normal_sequences:
    result = tsunami_detector.detect_tsunami(sequence)
    print(f"NORMAL: Is Tsunami: {result['is_tsunami']}, Confidence: {result['confidence']}, Anomaly Score: {result['anomaly_score']}")


NORMAL: Is Tsunami: True, Confidence: 1.0, Anomaly Score: 3.336440086364746
NORMAL: Is Tsunami: True, Confidence: 1.0, Anomaly Score: 3.471646785736084
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.2186741828918457
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.2597107887268066
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.2022886276245117
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.3016515970230103
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.3189741373062134
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.5805974006652832
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.915256142616272
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.654782772064209
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.0934669971466064
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.2396166324615479
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.748955249